# Gain calibration using SVD

**T. Carozzi**, *Onsala Space Observatory*

In [1]:

import numpy as np
import numpy.linalg as la
import mpl_toolkits as mpl

# Something wrong with path to mpl_toolkits, so hack it:
mpl.__path__ = ['/home/tobia/.local/lib/python3.8/site-packages/matplotlib-3.3.1-py3.8-linux-x86_64.egg/mpl_toolkits']
%matplotlib qt
import matplotlib.pyplot as plt

from ilisa.observations.dataIO import *
from ilisa.calim.imaging import *
from ilisa.calim.imageprocess import *
from ilisa.calim.calibration import apply_polgains, gain_cal_bs_lin
from ilisa.observations.directions import directionterm2tuple

Read in a visibility dataset specifyied by path:

In [3]:
datahome ='/home/tobia/Documents/Data/LOFAR_COPY/'
cvcpath_acc = datahome+'proj31/sess_sid20200422T080002_SE607/scan_58961.33361/SE607_20200422_080850_rcu3_dur512_acc/'
# 'sess_sid20201117T124331_SE607/scan_59170.53154/SE607_20201117_125349_rcu3_dur512_acc/'
# 'proj31/sess_sid20200422T080002_SE607/scan_58961.33361/SE607_20200422_080850_rcu3_dur512_acc/'
cvcpath_xst = '/home/tobia/Documents/Data/LOFAR_COPY/xst/scan_59052.40191/20200722_093847_rcu4_sb457_int1_dur3_dir0.,1.5707963267948966,AZELGEO_xst'
cvcpath = cvcpath_acc
polrep='stokes'
calsrc='Cas-A'
cvcds=CVCfiles(cvcpath)




Reading cvcfile: 20200422_080850_acc_512x192x192.dat
Reading cvcfile: 20200422_081729_acc_512x192x192.dat


In [4]:
filenr=0
cvpol = cvcds.covcube_fb(filenr)

sampnr=222 # 195, 222, 299
vis=cvpol[:,:,sampnr,:,:]

For this selecton, the observational parameters are:

In [5]:
t=cvcds.samptimeset[filenr][sampnr]
freq=cvcds.freqset[filenr][sampnr]
print("Time: {} UT, Frequency: {} MHz, PolarRep: {}".format(t, freq/1e6, polrep))

Time: 2020-04-22 08:04:00 UT, Frequency: 43.359375 MHz, PolarRep: stokes


## Looking at the visibilities

Make a beamformed image and plot it

In [6]:
uvw_zen = calc_uvw(t, directionterm2tuple('Z'), cvcds.stn_pos, cvcds.stn_antpos)
imgs_nocal, ll, mm = beamformed_image(vis, uvw_zen.T, freq, use_autocorr=False, polrep=polrep, fluxperbeam=True)
plotskyimage(ll, mm, imgs_nocal, polrep, t, freq, cvcds.scanrecinfo.stnid, cvcds.scanrecinfo.get_integration(),phaseref=directionterm2tuple('Z'))

In [7]:
from ilisa.calim.robust_cal import robcal

In [8]:
ginv_r=robcal(phasedup_vis(vis, calsrc, t, freq, polrep, cvcds.stn_pos, cvcds.stn_antpos))
ginv_r0=numpy.ones(ginv_r.shape)
vis_cal_r = apply_polgains(ginv_r, vis)

In [9]:
imgs_cal_r, ll, mm = beamformed_image(vis_cal_r, uvw_zen.T, freq, use_autocorr=False, polrep=polrep, fluxperbeam=True)
plotskyimage(ll,mm, imgs_cal_r, polrep, t, freq, cvcds.scanrecinfo.stnid, cvcds.scanrecinfo.get_integration(),phaseref=directionterm2tuple('Z'),calibrated=True)

## Compare with standard cal

In [10]:
ginv_s = gain_cal_bs_lin(phasedup_vis(vis, calsrc, t, freq, polrep, cvcds.stn_pos, cvcds.stn_antpos))
vis_cal_s = apply_polgains(1/ginv_s, vis)

In [11]:
imgs_cal_s, ll, mm = beamformed_image(vis_cal_s, uvw_zen.T, freq, use_autocorr=False, polrep=polrep, fluxperbeam=True)
plotskyimage(ll,mm, imgs_cal_s, polrep, t, freq, cvcds.scanrecinfo.stnid, cvcds.scanrecinfo.get_integration(),phaseref=directionterm2tuple('Z'))

In [12]:
def xtra_im_noise(img):
    from ilisa.calim.imageprocess import split_horizon
    a,b,h=split_horizon(img)
    return np.std(b)
def dyn_rng(img):
    noise=xtra_im_noise(img)
    return (np.max(img)-np.mean(img))/noise

In [13]:
dyn_rng(imgs_nocal[0]), dynamic_range(imgs_nocal[0])

(28.519004864437786, 9.170042554157906)

In [14]:
dyn_rng(imgs_cal_s[0]), dynamic_range(imgs_cal_s[0])

(30.168099448429558, 9.360209950022924)

In [15]:
dyn_rng(imgs_cal_r[0]), dynamic_range(imgs_cal_r[0])

(30.16818036169454, 9.36022630396379)

In [16]:
plt.subplot(131)
_=plt.hist(np.ravel(imgs_nocal[0]),100,log=True)
plt.subplot(132)
_=plt.hist(np.ravel(imgs_cal_s[0]),100,log=True)
plt.subplot(133)
_=plt.hist(np.ravel(imgs_cal_r[0]),100,log=True)
